In [1]:
!pip install llama-cpp-python langchain_community langchain_core huggingface_hub pypdf faiss-cpu duckduckgo-search langchain-huggingface  google-search-results

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 MB 9.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 73.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.7/407.7 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.5/294.5 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.9/296.9 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 84.0 MB/s et

In [2]:
from huggingface_hub import hf_hub_download

model_rpo = "GaoDalie/llama-3-8b-Instruct-bnb-4bit-gaodalie-demo"
model_file = "unsloth.Q4_K_M.gguf"

local_dir = "Models"

download_file = hf_hub_download(repo_id=model_rpo,filename=model_file,local_dir=local_dir)

print(f"model download to: {download_file}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


unsloth.Q4_K_M.gguf:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model download to: Models/unsloth.Q4_K_M.gguf


In [3]:
Model_Path = 'Models/unsloth.Q4_K_M.gguf'

In [4]:
import os
import sys
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
import json
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.llms import LlamaCpp
from typing import List , Tuple
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..'))) # Add the parent directory to the path sicnce we work with notebooks
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_huggingface import HuggingFaceEmbeddings

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
path = "/content/Lecture8.pdf"

In [6]:
embed_model = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
def replace_t_with_space(list_of_documents):
    """
    Replaces all tab characters ('\t') with spaces in the page content of each document.

    Args:
        list_of_documents: A list of document objects, each with a 'page_content' attribute.

    Returns:
        The modified list of documents with tab characters replaced by spaces.
    """

    for doc in list_of_documents:
        doc.page_content = doc.page_content.replace('\t', ' ')  # Replace tabs with spaces
    return list_of_documents

def encode_pdf(path, chunk_size=1000, chunk_overlap=200):
    """
    Encodes a PDF book into a vector store using OpenAI embeddings.

    Args:
        path: The path to the PDF file.
        chunk_size: The desired size of each text chunk.
        chunk_overlap: The amount of overlap between consecutive chunks.

    Returns:
        A FAISS vector store containing the encoded book content.
    """

    # Load PDF documents
    loader = PyPDFLoader(path)
    documents = loader.load()

    # Split documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap, length_function=len
    )
    texts = text_splitter.split_documents(documents)
    cleaned_texts = replace_t_with_space(texts)

    # Create embeddings and vector store
    embeddings = embed_model
    vectorstore = FAISS.from_documents(cleaned_texts, embeddings)

    return vectorstore

In [8]:
vectorstore = encode_pdf(path)

In [9]:
from langchain_community.chat_models import ChatLlamaCpp
llm = ChatLlamaCpp(
    model_path=Model_Path,
    temperature=0.75,
    max_tokens=2000,
    top_p=1,
)

llama_model_loader: loaded meta data with 27 key-value pairs and 291 tensors from Models/unsloth.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Llama 3 8b Instruct Bnb 4bit Gaodalie...
llama_model_loader: - kv   3:                           general.finetune str              = Instruct-bnb-4bit-gaodalie-demo
llama_model_loader: - kv   4:                           general.basename str              = llama-3
llama_model_loader: - kv   5:                         general.size_label str              = 8B
llama_model_loader: - kv   6:                          llama.block_count u32              = 32
llama_model_loader: - k

In [10]:
search = DuckDuckGoSearchResults()

In [11]:
from langchain_core.utils.function_calling import convert_to_openai_tool

In [12]:
# Retrieval Evaluator
class RetrievalEvaluatorInput(BaseModel):
    relevance_score: float = Field(..., description="The relevance score of the document to the query. the score should be between 0 and 1.")
def retrieval_evaluator(query: str, document: str) -> float:
    prompt = PromptTemplate(
        input_variables=["query", "document"],
        template="On a scale from 0 to 1, how relevant is the following document to the query? Query: {query}\nDocument: {document}\nRelevance score:"
    )
    dict_schema = convert_to_openai_tool(RetrievalEvaluatorInput)
    print("Prompt:", prompt)
    print("Schema:", dict_schema)
    chain = prompt | llm.with_structured_output(dict_schema)
    input_variables = {"query": query, "document": document}
    print("Input variables:", input_variables)
    result = chain.invoke(input_variables)
    print(f"Result from chain: {result}")
    relevance_score = result['relevance_score']
    return relevance_score

# Knowledge Refinement
class KnowledgeRefinementInput(BaseModel):
    key_points: str = Field(..., description="The document to extract key information from.")
def knowledge_refinement(document: str) -> List[str]:
    prompt = PromptTemplate(
        input_variables=["document"],
        template="Extract the key information from the following document in bullet points:\n{document}\nKey points:"
    )
    chain = prompt | llm.with_structured_output(KnowledgeRefinementInput)
    input_variables = {"document": document}
    result = chain.invoke(input_variables).key_points
    return [point.strip() for point in result.split('\n') if point.strip()]

# Web Search Query Rewriter
class QueryRewriterInput(BaseModel):
    query: str = Field(..., description="The query to rewrite.")
def rewrite_query(query: str) -> str:
    prompt = PromptTemplate(
        input_variables=["query"],
        template="Rewrite the following query to make it more suitable for a web search:\n{query}\nRewritten query:"
    )
    chain = prompt | llm.with_structured_output(QueryRewriterInput)
    input_variables = {"query": query}
    return chain.invoke(input_variables).query.strip()

In [13]:
def parse_search_results(results_string: str) -> List[Tuple[str, str]]:
    """
    Parse a JSON string of search results into a list of title-link tuples.
    """
    try:
        # Attempt to parse the JSON string
        results = json.loads(results_string)
        print("Parsed search results:", results)  # Debugging line
        # Extract and return the title and link from each result
        return [(result.get('title', 'Untitled'), result.get('link', '')) for result in results]
    except json.JSONDecodeError as e:
        # Handle JSON decoding errors by returning an empty list
        print(f"Error parsing search results: {e}. Returning empty list.")
        return []

In [14]:
api_key = 

In [15]:
from langchain_core.tools import Tool
from langchain_community.utilities import SerpAPIWrapper

# Initialize the SerpAPIWrapper with your API key
search = SerpAPIWrapper(serpapi_api_key=api_key)


def top5_results(query: str) -> List:
    """
    Retrieve the search results for a given query
    and return them directly.

    Args:
        query (str): The search query to execute.

    Returns:
        List: The search results returned by the SerpAPIWrapper.
    """
    # Use the run method and return the results directly
    results = search.run(query)  # Pass only the query
    print("Retrieved search results:", results)  # Debugging line
    return results  # Return the raw results

# Define the tool
tool = Tool(
    name="Google Search Snippets",
    description="Search Google for recent results.",
    func=top5_results,
)

In [16]:
def retrieve_documents(query: str, faiss_index: FAISS, k: int = 3) -> List[str]:
    """
    Retrieve documents based on a query using a FAISS index.

    Args:
        query (str): The query string to search for.
        faiss_index (FAISS): The FAISS index used for similarity search.
        k (int): The number of top documents to retrieve. Defaults to 3.

    Returns:
        List[str]: A list of the retrieved document contents.
    """
    docs = faiss_index.similarity_search(query, k=k)
    return [doc.page_content for doc in docs]

def evaluate_documents(query: str, documents: List[str]) -> List[float]:
    """
    Evaluate the relevance of documents based on a query.

    Args:
        query (str): The query string.
        documents (List[str]): A list of document contents to evaluate.

    Returns:
        List[float]: A list of relevance scores for each document.
    """
    return [retrieval_evaluator(query, doc) for doc in documents]

def perform_web_search(query: str) -> Tuple[List[str], List[Tuple[str, str]]]:
    """
    Perform a web search based on a query.
    """
    rewritten_query = rewrite_query(query)
    print(f"Performing web search for query: {rewritten_query}")  # Debugging line

    try:
        web_results = tool.run(rewritten_query)  # Only pass the rewritten query
        print("Web results string:", web_results)  # Debugging line
        web_knowledge = knowledge_refinement(web_results)
        print("Refined knowledge from web results:", web_knowledge)  # Debugging line

        return web_knowledge, web_results
    except Exception as e:
        print(f"Error during web search: {e}")
        return [], []

def generate_response(query: str, knowledge: str) -> str:
    """
    Generate a response to a query using knowledge and sources.

    Args:
        query (str): The query string.
        knowledge (str): The refined knowledge to use in the response.
        sources (List[Tuple[str, str]]): A list of tuples containing titles and links of the sources.

    Returns:
        str: The generated response.
    """
    response_prompt = PromptTemplate(
        input_variables=["query", "knowledge"],
        template="Based on the following knowledge, answer the query. Include the sources with their links (if available) at the end of your answer:\nQuery: {query}\nKnowledge: {knowledge}\nAnswer:"
    )
    input_variables = {
        "query": query,
        "knowledge": knowledge,
    }
    response_chain = response_prompt | llm
    return response_chain.invoke(input_variables).content

In [17]:
def crag_process(query: str, faiss_index: FAISS) -> str:
    """
    Process a query by retrieving, evaluating, and using documents or performing a web search to generate a response.

    Args:
        query (str): The query string to process.
        faiss_index (FAISS): The FAISS index used for document retrieval.

    Returns:
        str: The generated response based on the query.
    """
    print(f"\nProcessing query: {query}")

    # Retrieve and evaluate documents
    retrieved_docs = retrieve_documents(query, faiss_index)
    eval_scores = evaluate_documents(query, retrieved_docs)

    print(f"\nRetrieved {len(retrieved_docs)} documents")
    print(f"Evaluation scores: {eval_scores}")

    # Determine action based on evaluation scores
    max_score = max(eval_scores)
    sources = []

    if max_score > 0.7:
        print("\nAction: Correct - Using retrieved document")
        best_doc = retrieved_docs[eval_scores.index(max_score)]
        final_knowledge = best_doc
        sources.append(("Retrieved document", ""))
    elif max_score < 0.3:
        print("\nAction: Incorrect - Performing web search")
        final_knowledge, sources = perform_web_search(query)
    else:
        print("\nAction: Ambiguous - Combining retrieved document and web search")
        best_doc = retrieved_docs[eval_scores.index(max_score)]
        # Refine the retrieved knowledge
        retrieved_knowledge = knowledge_refinement(best_doc)
        web_knowledge, web_sources = perform_web_search(query)
        final_knowledge = "\n".join(retrieved_knowledge + web_knowledge)
        sources = [("Retrieved document", "")] + web_sources

    print("\nFinal knowledge:")
    print(final_knowledge)

    # Generate response
    print("\nGenerating response...")
    response = generate_response(query, final_knowledge)

    print("\nResponse generated")
    return response

In [18]:
query = "What are the sites of protein synthesis or assembly?"
result = crag_process(query, vectorstore)
print(f"Query: {query}")
print(f"Answer: {result}")


Processing query: What are the sites of protein synthesis or assembly?
Prompt: input_variables=['document', 'query'] input_types={} partial_variables={} template='On a scale from 0 to 1, how relevant is the following document to the query? Query: {query}\nDocument: {document}\nRelevance score:'
Schema: {'type': 'function', 'function': {'name': 'RetrievalEvaluatorInput', 'description': '', 'parameters': {'type': 'object', 'properties': {'relevance_score': {'description': 'The relevance score of the document to the query. the score should be between 0 and 1.', 'type': 'number'}}, 'required': ['relevance_score']}}}
Input variables: {'query': 'What are the sites of protein synthesis or assembly?', 'document': 'Fleck, Ludwik  (1975), Genesis and Development of a Scientific Fact , Univ. of Chicago,  \n(written in German, 1935, Entstehung und Entwickelung einer wissenschaftlichen Tatsache: \nEinführung in die Lehre vom Denkstil und Denkkollectiv ) English translation  \nGauch, Hugh G., Jr. (

llama_perf_context_print:        load time =  151336.20 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   283 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    12 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =  160930.05 ms /   295 tokens
Llama.generate: 39 prefix-match hit, remaining 208 prompt tokens to eval


Result from chain: {'relevance_score': 0.01}
Prompt: input_variables=['document', 'query'] input_types={} partial_variables={} template='On a scale from 0 to 1, how relevant is the following document to the query? Query: {query}\nDocument: {document}\nRelevance score:'
Schema: {'type': 'function', 'function': {'name': 'RetrievalEvaluatorInput', 'description': '', 'parameters': {'type': 'object', 'properties': {'relevance_score': {'description': 'The relevance score of the document to the query. the score should be between 0 and 1.', 'type': 'number'}}, 'required': ['relevance_score']}}}
Input variables: {'query': 'What are the sites of protein synthesis or assembly?', 'document': "Since it is impossible for a s cientist to record everything  that took place in an experiment, \nfacts selected for their apparent relevance are reported . This may lead, unavoidably, to \nproblems later if some supposedly irrelevant feature is questioned. For example, Heinrich \nHertz  did not report the si

llama_perf_context_print:        load time =  151336.20 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   208 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    12 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =  113367.31 ms /   220 tokens
Llama.generate: 39 prefix-match hit, remaining 191 prompt tokens to eval


Result from chain: {'relevance_score': 0.1}
Prompt: input_variables=['document', 'query'] input_types={} partial_variables={} template='On a scale from 0 to 1, how relevant is the following document to the query? Query: {query}\nDocument: {document}\nRelevance score:'
Schema: {'type': 'function', 'function': {'name': 'RetrievalEvaluatorInput', 'description': '', 'parameters': {'type': 'object', 'properties': {'relevance_score': {'description': 'The relevance score of the document to the query. the score should be between 0 and 1.', 'type': 'number'}}, 'required': ['relevance_score']}}}
Input variables: {'query': 'What are the sites of protein synthesis or assembly?', 'document': 'vary the conditions for each me asurement; to help isolate what has changed. Depending on \nthe predictions, the experiments can have different shapes. It could be a classical experiment \nin a laboratory setting, a  double -blind  study or an archaeological excavation .  \nScientists assume an attitude of ope

...

Performing web search for query: Where do proteins get made or assembled? Sites of protein synthesis or assembly.
Retrieved search results: To maintain the correct reading frame and to ensure accuracy (about 1 mistake every 10,000 amino acids), protein synthesis is performed in the ribosome, a complex catalytic machine made from more than 50 different proteins (the ribosomal proteins) and several RNA molecules, the ribosomal RNAs (rRNAs).
Web results string: To maintain the correct reading frame and to ensure accuracy (about 1 mistake every 10,000 amino acids), protein synthesis is performed in the ribosome, a complex catalytic machine made from more than 50 different proteins (the ribosomal proteins) and several RNA molecules, the ribosomal RNAs (rRNAs).


Llama.generate: 5 prefix-match hit, remaining 86 prompt tokens to eval
llama_perf_context_print:        load time =  151336.20 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    86 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   193 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =  190786.72 ms /   279 tokens
Llama.generate: 5 prefix-match hit, remaining 234 prompt tokens to eval


Refined knowledge from web results: ['Protein synthesis in the ribosome is a complex process that requires accuracy to ensure proper protein function. The following key points highlight the importance of this process:', '• Protein synthesis occurs in the ribosome, a complex catalytic machine made up of more than 50 different proteins and several RNA molecules.', '• To maintain the correct reading frame and to ensure accuracy, protein synthesis is performed in the ribosome.', '• The ribosome is responsible for translating messenger RNA (mRNA) into a sequence of amino acids that make up a protein.', '• Protein synthesis involves three stages: initiation, elongation, and termination. Each stage requires specific proteins and RNAs to function correctly.', '• To ensure accuracy during protein synthesis, the ribosome uses several mechanisms to proofread and edit the growing polypeptide chain.', '• The final product of protein synthesis is a mature protein that can perform its intended biolog

llama_perf_context_print:        load time =  151336.20 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   234 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   162 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =  230579.80 ms /   396 tokens



Response generated
Query: What are the sites of protein synthesis or assembly?
Answer: Based on the provided knowledge, protein synthesis occurs in the ribosome, a complex catalytic machine made up of more than 50 different proteins and several RNA molecules.

The sites of protein synthesis or assembly are:

1. The ribosome: This is where the genetic information encoded in messenger RNA (mRNA) is translated into a sequence of amino acids that make up a protein.
2. The endoplasmic reticulum (ER): This is where proteins synthesized by the ribosome are folded and modified to become mature proteins.
3. The Golgi apparatus: This is where proteins synthesized by the ER are further processed, packaged, and shipped to their final destinations within the cell.

These sites of protein synthesis or assembly are crucial for the proper functioning of cells, tissues, and organs in living organisms.
